In [35]:
import pandas as pd
ac = pd.read_csv('../data/raw/ncaa_accolades_2004_to_2023.csv')
ac = ac.drop(columns=['Division', 'INSTNM', 'INSTNM_NORM', 'Champion_NORM', 'Champion', 'type_of_match'])
ac

,Gender,Sport,Year,UNITID
0,Men,Wrestling,2023,214777
1,Men,Wrestling,2022,214777
2,Men,Wrestling,2021,153658
3,Men,Wrestling,2019,214777
4,Men,Wrestling,2018,214777
...,...,...,...,...
1118,Men,Football,2008,233374
1119,Men,Football,2007,197869
1120,Men,Football,2006,197869
1121,Men,Football,2005,197869


In [36]:
# Group by UNITID, Year, Gender and count championships
df_summary = (
    ac.groupby(['UNITID', 'Year', 'Gender'])
      .size()  # counts the number of rows in each group
      .reset_index(name='Number of Championships')
)

# Optional: sort by UNITID then Year
df_summary = df_summary.sort_values(by=['UNITID', 'Year']).reset_index(drop=True)
df_summary.sort_values('Number of Championships', ascending=False)

,UNITID,Year,Gender,Number of Championships
1007,243744,2019,Women,4
467,168342,2015,Women,4
68,110662,2004,Women,4
954,240189,2014,Men,3
4,100751,2012,Women,3
...,...,...,...,...
362,154527,2005,Women,1
363,154527,2006,Men,1
364,154527,2008,Men,1
365,154527,2009,Men,1


In [37]:
ac['Gender'].value_counts()

Gender
Men      586
Women    537
Name: count, dtype: int64

In [38]:
ac['Gender and Sport'] = ac['Gender'] + "'s " + ac['Sport'] + " Champion"
df = ac.drop(columns=['Gender', 'Sport'])
print(df)

      Year  UNITID          Gender and Sport
0     2023  214777  Men's Wrestling Champion
1     2022  214777  Men's Wrestling Champion
2     2021  153658  Men's Wrestling Champion
3     2019  214777  Men's Wrestling Champion
4     2018  214777  Men's Wrestling Champion
...    ...     ...                       ...
1118  2008  233374   Men's Football Champion
1119  2007  197869   Men's Football Champion
1120  2006  197869   Men's Football Champion
1121  2005  197869   Men's Football Champion
1122  2004  232423   Men's Football Champion

[1123 rows x 3 columns]


In [39]:
one_hot = (
    df.pivot_table(
        index=["Year", "UNITID"],
        columns="Gender and Sport",
        values="Gender and Sport",
        aggfunc=lambda x: 1
    )
    .fillna(0)
    .astype(int)
    .reset_index()
)
one_hot['Total Champions'] = one_hot.iloc[:, 2:].sum(axis=1)
one_hot

Gender and Sport,Year,UNITID,Men's Baseball Champion,Men's Basketball Champion,Men's Cross Country Champion,Men's Football Champion,Men's Golf Champion,Men's Gymnastics Champion,Men's Indoor Track & Field Champion,Men's Lacrosse Champion,...,Women's Gymnastics Champion,Women's Indoor Track & Field Champion,Women's Lacrosse Champion,Women's Outdoor Track & Field Champion,Women's Soccer Champion,Women's Softball Champion,Women's Swimming & Diving Champion,Women's Tennis Champion,Women's Volleyball Champion,Total Champions
0,2004,100858,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,2
1,2004,106245,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2004,110635,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2004,110662,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,4
4,2004,119058,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,2023,232609,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
903,2023,234076,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,2
904,2023,240329,0,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,3
905,2023,243744,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [40]:
one_hot.to_pickle('../data/processed/Accolades.pkl')

In [41]:
import pickle

champ_cols = list(one_hot.columns[2:])
pickle.dump(champ_cols, open('../data/mappings/Accolades_columns.pkl', 'wb'))